<a href="https://colab.research.google.com/github/newmantic/LIME/blob/main/LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import sklearn
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.utils import check_random_state
import matplotlib.pyplot as plt

In [17]:
class LimeExplainer:
    def __init__(self, kernel_width=0.75):
        self.kernel_width = kernel_width
        self.scaler = None

    def kernel(self, distances):
        """
        Exponential kernel to give more weight to points closer to the original instance.
        """
        return np.sqrt(np.exp(-(distances ** 2) / self.kernel_width ** 2))

    def generate_binary_perturbations(self, instance, num_samples=5000):
        """
        Generate binary perturbations where each feature is either kept as is or turned off.
        """
        np.random.seed(42)
        # Binary perturbation: either keep the feature or set to zero
        perturbations = np.random.randint(0, 2, (num_samples, instance.shape[1]))
        perturbed_data = perturbations * instance  # Apply binary mask to original instance

        # Compute distances and apply kernel
        distances = pairwise_distances(perturbed_data, instance, metric='euclidean').ravel()
        weights = self.kernel(distances)
        return perturbed_data, weights

    def fit_local_model(self, perturbations, predictions, weights):
        """
        Fit a Ridge regression model locally on the perturbations.
        """
        # Fit a simple Ridge model using perturbations and their weights
        model = Ridge(alpha=1, fit_intercept=True)
        model.fit(perturbations, predictions, sample_weight=weights)
        return model

    def explain(self, model, instance, num_samples=5000):
        """
        Explain the predictions of the given model for a specific instance using LIME.
        """
        # Scale instance if scaler is fitted
        if self.scaler is not None:
            instance = self.scaler.transform(instance.reshape(1, -1))

        perturbations, weights = self.generate_binary_perturbations(instance, num_samples=num_samples)
        model_predictions = model.predict(perturbations)

        # Fit local model and extract feature importance
        local_model = self.fit_local_model(perturbations, model_predictions, weights)
        explanation = local_model.coef_
        return explanation

    def fit_scaler(self, X_train):
        """
        Fit a scaler to normalize the data.
        """
        self.scaler = StandardScaler().fit(X_train)

In [19]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Load dataset and train a model
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Fit the scaler on the training data
explainer = LimeExplainer(kernel_width=0.5)
explainer.fit_scaler(X_train)

# Select an instance to explain
instance = X_test[0].reshape(1, -1)

# Explain the prediction for the instance
explanation = explainer.explain(model, instance)

# Print feature importance for the instance
print("Feature importance for the instance:")
for i, feature in enumerate(iris.feature_names):
    print(f"{feature}: {explanation[i]:.4f}")

Feature importance for the instance:
sepal length (cm): 0.0000
sepal width (cm): 0.0000
petal length (cm): 0.0000
petal width (cm): 0.0000


In [18]:
from sklearn.neural_network import MLPClassifier

# Train a neural network model
nn_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, random_state=42)
nn_model.fit(X_train, y_train)

# Explain the prediction of the neural network
nn_explanation = explainer.explain(nn_model, instance)

# Print the explanation for the neural network model
print("\nNeural Network - Feature importance for the instance:")
for i, feature in enumerate(iris.feature_names):
    print(f"{feature}: {nn_explanation[i]:.4f}")


Neural Network - Feature importance for the instance:
sepal length (cm): -0.6042
sepal width (cm): -0.8378
petal length (cm): 0.8479
petal width (cm): 2.0943
